In [1]:
#importing neccesary libraries
import pandas as pd
import requests
import numpy as np

In [2]:
#getting the dataset from the github link given
url = 'https://raw.githubusercontent.com/WalePhenomenon/climate_change/master/fuel_ferc1.csv'
request = requests.get(url)
content_of_url = request.content
data_csv = open('dataset.csv', 'wb')
data_csv.write(content_of_url)
data_csv.close()

In [3]:
#reading the dataset with pandas
data_hamoye = pd.read_csv('dataset.csv')

# Question 1

# Question 2

# Question 3

In [4]:
def max_fuel_cost(df, cat_column):
    """This function takes a dataframe and a string of categorical column"""
    data = data_hamoye.groupby(cat_column)
    #defining the aggregation dictionary of the function we want to perform in groups
    aggregation = ({'fuel_cost_per_unit_burned': 'mean'})
    #implenting the aggregation
    max_fuel = data.agg(aggregation)
    return max_fuel.nsmallest(1, 'fuel_cost_per_unit_burned')

In [5]:
max_fuel_cost(data_hamoye, 'fuel_type_code_pudl')

,fuel_cost_per_unit_burned
fuel_type_code_pudl,
gas,13.659397


# Question 4

In [6]:
def std_perc(df, column):
    """This function will take a dataframe and a particular string of column name
    it will return the standard deviation and 75th percentile for the column
    """
    std_col = df.loc[:, column].std()
    perc_col = np.percentile(df.loc[:, column], 75)
    return (f"standard deviation is:{round(std_col, 2)} and 75th percentile is: {round(perc_col, 2)}")

In [7]:
std_perc(data_hamoye, 'fuel_mmbtu_per_unit')

'standard deviation is:10.6 and 75th percentile is: 17.01'

# Question 5

In [8]:
def kurt_skew(df, num_column):
    """This function takes a dataframe and a string of numerical column
    it will return the kurtosis and the skewness of the numerical column
    """
    kurt_col = df.loc[:, num_column].kurt()
    skew_col = df.loc[:, num_column].skew()
    return(f"The kurtosis is: {round(kurt_col, 2)} and the skewness is: {round(skew_col, 2)}")
    

In [9]:
kurt_skew(data_hamoye, 'fuel_qty_burned')

'The kurtosis is: 651.37 and the skewness is: 15.85'

# Question 6

In [13]:
def missing_values(dataframe):
    """
    This function takes a dataframe as input and returns two columns... total missing values and their percentage
    """
    total = dataframe.isnull().sum().sort_values(ascending = False)
    percent = round(total/len(dataframe)*100, 3)
    return pd.concat([total, percent], axis = 1, keys = ['Total Missing', '%Percentage'])

In [14]:
df = missing_values(data_hamoye)
df.head()

,Total Missing,%Percentage
fuel_unit,180,0.61
fuel_cost_per_mmbtu,0,0.00
fuel_cost_per_unit_delivered,0,0.00
fuel_cost_per_unit_burned,0,0.00
fuel_mmbtu_per_unit,0,0.00


In [15]:
df.iloc[0]

Total Missing    180.00
%Percentage        0.61
Name: fuel_unit, dtype: float64

# Question 8

In [18]:
corr_num = data_hamoye.corr()
print('features with the lowest correlation with the fuel_cost_per_unit_burned : ')
print(corr_num.nsmallest(3, 'fuel_cost_per_unit_burned')['fuel_cost_per_unit_burned'])

features with the lowest correlation with the fuel_cost_per_unit_burned : 
utility_id_ferc1      -0.037863
fuel_qty_burned       -0.018535
fuel_mmbtu_per_unit   -0.010034
Name: fuel_cost_per_unit_burned, dtype: float64


# Question 9

In [20]:
#lets make an aggregation dictionary of functions we want to perform in groups 
aggregations = ({'fuel_cost_per_unit_burned': sum,})#mum of fuel_cost_per_unit_burned in each group
#getting the groups
perc = data_hamoye.groupby(['report_year', 'fuel_type_code_pudl'])
perc_a = perc.get_group((1998, 'coal')) #i will use this one to get the total sum of fcpbb belongin to coal in 1998
perc_b = perc.get_group((1994, 'coal')) #i will use this to find the total sum of fcpb belonging to  in 1994

#applying aggregation to each of the groups
perc_a1 = perc_a.agg(aggregations)#will give the sum of qty for 114 in 1994
perc_b1 = perc_b.agg(aggregations)

### Recall that percentage change is given by $\text{percent change} = \frac{\text{New value} - \text{old value}}{\text{old value}}\times 100$

In [21]:
perc_diff = ((perc_a1 - perc_b1)/perc_b1)*100
perc_diff

fuel_cost_per_unit_burned   -20.567655
dtype: float64

# Question 10

In [25]:
#getting the groups
rep = data_hamoye.groupby(['report_year'])
aggregations = ({'fuel_cost_per_unit_delivered': 'mean',})
hig = rep.agg(aggregations)
highest_fuel = hig.nlargest(1, 'fuel_cost_per_unit_delivered')
highest_fuel

,fuel_cost_per_unit_delivered
report_year,
1997,11140.197239
